In [2]:
import os
import cv2
import os.path
import sqlite3
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, transform
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.optimizers import RMSprop
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
print(os.listdir("../input/aflw/AFLW"))

Using TensorFlow backend.


['aflw-images', 'aflw_example.py', 'gui', 'matlab', 'aflw-gui-bin-win32', 'aflw-db', 'facedbsql']


In [3]:
%%time
#Code from AFLW webiste to connect to database

#Change this paths according to your directories
images_path = "../input/aflw/AFLW/aflw-images/data/flickr/"
#storing_path = "./output/"

images = []
landmarks = []
visibility = []
pose = []
gender = []


#Image counter
counter = 1

#Open the sqlite database
conn = sqlite3.connect('../input/aflw/AFLW/aflw-db/data/aflw.sqlite')
c = conn.cursor()

#Creating the query string for retriving: roll, pitch, yaw and faces position
#Change it according to what you want to retrieve
select_string = "faceimages.filepath, faces.face_id, facepose.roll, facepose.pitch, facepose.yaw, facerect.x, facerect.y, facerect.w, facerect.h, facemetadata.sex"
from_string = "faceimages, faces, facepose, facerect, facemetadata"
where_string = "faces.face_id = facepose.face_id and faces.file_id = faceimages.file_id and faces.face_id = facerect.face_id and faces.face_id = facemetadata.face_id"
query_string = "SELECT " + select_string + " FROM " + from_string + " WHERE " + where_string

#It iterates through the rows returned from the query
for row in c.execute(query_string):

    #Using our specific query_string, the "row" variable will contain:
    # row[0] = image path
    # row[1] = face id
    # row[2] = roll
    # row[3] = pitch
    # row[4] = yaw
    # row[5] = face coord x
    # row[6] = face coord y
    # row[7] = face width
    # row[8] = face heigh
    # row[9] = sex

    #Creating the full path names for input and output
    input_path = images_path + str(row[0])
    #output_path = storing_path + str(row[0])

    #If the file exist then open it       
    if(os.path.isfile(input_path)  == True):
        image = cv2.imread(input_path, 1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #load the colour version

        #Image dimensions
        image_h, image_w = image.shape
        #Roll, pitch and yaw
        roll   = row[2]
        pitch  = row[3]
        yaw    = row[4]
        #Face rectangle coords
        face_x = row[5]
        face_y = row[6]
        face_w = row[7]
        face_h = row[8]
        #Gender
        sex = ( 1 if row[9] == 'm' else 0 )
        
        gender.append(sex)
        pose.append(np.asarray([roll, pitch, yaw]))
        
        
        select_str = "coords.feature_id, coords.x, coords.y"
        from_str = "featurecoords coords"
        where_str = "coords.face_id = {}".format(row[1])
        query_str = "SELECT " + select_str + " FROM " + from_str + " WHERE " + where_str
        lm = np.zeros((21,2)).astype(np.float32)
        v = np.zeros((21)).astype(np.int32)
        
        c2 = conn.cursor()
        
        for q in c2.execute(query_str):
            lm[q[0]-1][0] = q[1]
            lm[q[0]-1][1] = q[2]
            v[q[0]-1] = 1
        
        lm = lm.reshape(42)
        
        landmarks.append(lm)
        visibility.append(v)

        c2.close()
        
        
        #Error correction
        #if(face_x < 0): face_x = 0
        #if(face_y < 0): face_y = 0
        #if(face_w > image_w): 
        #   face_w = image_w
        #   face_h = image_w
        #if(face_h > image_h): 
        #   face_h = image_h
        #   face_w = image_h

        #Crop the face from the image
        #image_cropped = np.copy(image[face_y:face_y+face_h, face_x:face_x+face_w])
        #Uncomment the lines below if you want to rescale the image to a particular size
        to_size = 227
        image_rescaled = cv2.resize(image, (to_size,to_size))
        #Uncomment the line below if you want to use adaptive histogram normalisation
        #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
        #image_normalised = clahe.apply(image_rescaled)
        #Save the image
        #change "image_cropped" with the last uncommented variable name above
        #cv2.imwrite(output_path, image_cropped)
        images.append(image_rescaled)
        

        #Printing the information
        print("Counter: " + str(counter))
        #print("iPath:    " + input_path)
        #print("oPath:    " + output_path)
        #print("x:       " + str(face_x))
        #print("y:       " + str(face_y))
        #print("w:       " + str(face_w))
        #print("h:       " + str(face_h))
        #print("Gender:  " + str(sex))
        #print("Pose:    " + str(np.asarray([roll, pitch, yaw])))
        #print("Image shape: " + str(image_rescaled.shape))
        #print("Face coord shape: " + str(lm.shape))
        #print("Visibility shape: " + str(v.shape))
        #print("")
        #Increasing the counter
        counter = counter + 1
            #if the file does not exits it return an exception
    else:
        #raise ValueError('Error: I cannot find the file specified: ' + str(input_path))
        continue

        #Once finished the iteration it closes the database
c.close()

print("Image list items: ", len(images))
print("Pose list items: ", len(pose))
print("Gender list items: ", len(gender))

Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Counter: 6
Counter: 7
Counter: 8
Counter: 9
Counter: 10
Counter: 11
Counter: 12
Counter: 13
Counter: 14
Counter: 15
Counter: 16
Counter: 17
Counter: 18
Counter: 19
Counter: 20
Counter: 21
Counter: 22
Counter: 23
Counter: 24
Counter: 25
Counter: 26
Counter: 27
Counter: 28
Counter: 29
Counter: 30
Counter: 31
Counter: 32
Counter: 33
Counter: 34
Counter: 35
Counter: 36
Counter: 37
Counter: 38
Counter: 39
Counter: 40
Counter: 41
Counter: 42
Counter: 43
Counter: 44
Counter: 45
Counter: 46
Counter: 47
Counter: 48
Counter: 49
Counter: 50
Counter: 51
Counter: 52
Counter: 53
Counter: 54
Counter: 55
Counter: 56
Counter: 57
Counter: 58
Counter: 59
Counter: 60
Counter: 61
Counter: 62
Counter: 63
Counter: 64
Counter: 65
Counter: 66
Counter: 67
Counter: 68
Counter: 69
Counter: 70
Counter: 71
Counter: 72
Counter: 73
Counter: 74
Counter: 75
Counter: 76
Counter: 77
Counter: 78
Counter: 79
Counter: 80
Counter: 81
Counter: 82
Counter: 83
Counter: 84
C

Counter: 644
Counter: 645
Counter: 646
Counter: 647
Counter: 648
Counter: 649
Counter: 650
Counter: 651
Counter: 652
Counter: 653
Counter: 654
Counter: 655
Counter: 656
Counter: 657
Counter: 658
Counter: 659
Counter: 660
Counter: 661
Counter: 662
Counter: 663
Counter: 664
Counter: 665
Counter: 666
Counter: 667
Counter: 668
Counter: 669
Counter: 670
Counter: 671
Counter: 672
Counter: 673
Counter: 674
Counter: 675
Counter: 676
Counter: 677
Counter: 678
Counter: 679
Counter: 680
Counter: 681
Counter: 682
Counter: 683
Counter: 684
Counter: 685
Counter: 686
Counter: 687
Counter: 688
Counter: 689
Counter: 690
Counter: 691
Counter: 692
Counter: 693
Counter: 694
Counter: 695
Counter: 696
Counter: 697
Counter: 698
Counter: 699
Counter: 700
Counter: 701
Counter: 702
Counter: 703
Counter: 704
Counter: 705
Counter: 706
Counter: 707
Counter: 708
Counter: 709
Counter: 710
Counter: 711
Counter: 712
Counter: 713
Counter: 714
Counter: 715
Counter: 716
Counter: 717
Counter: 718
Counter: 719
Counter: 720

Counter: 1268
Counter: 1269
Counter: 1270
Counter: 1271
Counter: 1272
Counter: 1273
Counter: 1274
Counter: 1275
Counter: 1276
Counter: 1277
Counter: 1278
Counter: 1279
Counter: 1280
Counter: 1281
Counter: 1282
Counter: 1283
Counter: 1284
Counter: 1285
Counter: 1286
Counter: 1287
Counter: 1288
Counter: 1289
Counter: 1290
Counter: 1291
Counter: 1292
Counter: 1293
Counter: 1294
Counter: 1295
Counter: 1296
Counter: 1297
Counter: 1298
Counter: 1299
Counter: 1300
Counter: 1301
Counter: 1302
Counter: 1303
Counter: 1304
Counter: 1305
Counter: 1306
Counter: 1307
Counter: 1308
Counter: 1309
Counter: 1310
Counter: 1311
Counter: 1312
Counter: 1313
Counter: 1314
Counter: 1315
Counter: 1316
Counter: 1317
Counter: 1318
Counter: 1319
Counter: 1320
Counter: 1321
Counter: 1322
Counter: 1323
Counter: 1324
Counter: 1325
Counter: 1326
Counter: 1327
Counter: 1328
Counter: 1329
Counter: 1330
Counter: 1331
Counter: 1332
Counter: 1333
Counter: 1334
Counter: 1335
Counter: 1336
Counter: 1337
Counter: 1338
Counte

Counter: 1860
Counter: 1861
Counter: 1862
Counter: 1863
Counter: 1864
Counter: 1865
Counter: 1866
Counter: 1867
Counter: 1868
Counter: 1869
Counter: 1870
Counter: 1871
Counter: 1872
Counter: 1873
Counter: 1874
Counter: 1875
Counter: 1876
Counter: 1877
Counter: 1878
Counter: 1879
Counter: 1880
Counter: 1881
Counter: 1882
Counter: 1883
Counter: 1884
Counter: 1885
Counter: 1886
Counter: 1887
Counter: 1888
Counter: 1889
Counter: 1890
Counter: 1891
Counter: 1892
Counter: 1893
Counter: 1894
Counter: 1895
Counter: 1896
Counter: 1897
Counter: 1898
Counter: 1899
Counter: 1900
Counter: 1901
Counter: 1902
Counter: 1903
Counter: 1904
Counter: 1905
Counter: 1906
Counter: 1907
Counter: 1908
Counter: 1909
Counter: 1910
Counter: 1911
Counter: 1912
Counter: 1913
Counter: 1914
Counter: 1915
Counter: 1916
Counter: 1917
Counter: 1918
Counter: 1919
Counter: 1920
Counter: 1921
Counter: 1922
Counter: 1923
Counter: 1924
Counter: 1925
Counter: 1926
Counter: 1927
Counter: 1928
Counter: 1929
Counter: 1930
Counte

Counter: 2474
Counter: 2475
Counter: 2476
Counter: 2477
Counter: 2478
Counter: 2479
Counter: 2480
Counter: 2481
Counter: 2482
Counter: 2483
Counter: 2484
Counter: 2485
Counter: 2486
Counter: 2487
Counter: 2488
Counter: 2489
Counter: 2490
Counter: 2491
Counter: 2492
Counter: 2493
Counter: 2494
Counter: 2495
Counter: 2496
Counter: 2497
Counter: 2498
Counter: 2499
Counter: 2500
Counter: 2501
Counter: 2502
Counter: 2503
Counter: 2504
Counter: 2505
Counter: 2506
Counter: 2507
Counter: 2508
Counter: 2509
Counter: 2510
Counter: 2511
Counter: 2512
Counter: 2513
Counter: 2514
Counter: 2515
Counter: 2516
Counter: 2517
Counter: 2518
Counter: 2519
Counter: 2520
Counter: 2521
Counter: 2522
Counter: 2523
Counter: 2524
Counter: 2525
Counter: 2526
Counter: 2527
Counter: 2528
Counter: 2529
Counter: 2530
Counter: 2531
Counter: 2532
Counter: 2533
Counter: 2534
Counter: 2535
Counter: 2536
Counter: 2537
Counter: 2538
Counter: 2539
Counter: 2540
Counter: 2541
Counter: 2542
Counter: 2543
Counter: 2544
Counte

...

Counter: 21900
Counter: 21901
Counter: 21902
Counter: 21903
Counter: 21904
Counter: 21905
Counter: 21906
Counter: 21907
Counter: 21908
Counter: 21909
Counter: 21910
Counter: 21911
Counter: 21912
Counter: 21913
Counter: 21914
Counter: 21915
Counter: 21916
Counter: 21917
Counter: 21918
Counter: 21919
Counter: 21920
Counter: 21921
Counter: 21922
Counter: 21923
Counter: 21924
Counter: 21925
Counter: 21926
Counter: 21927
Counter: 21928
Counter: 21929
Counter: 21930
Counter: 21931
Counter: 21932
Counter: 21933
Counter: 21934
Counter: 21935
Counter: 21936
Counter: 21937
Counter: 21938
Counter: 21939
Counter: 21940
Counter: 21941
Counter: 21942
Counter: 21943
Counter: 21944
Counter: 21945
Counter: 21946
Counter: 21947
Counter: 21948
Counter: 21949
Counter: 21950
Counter: 21951
Counter: 21952
Counter: 21953
Counter: 21954
Counter: 21955
Counter: 21956
Counter: 21957
Counter: 21958
Counter: 21959
Counter: 21960
Counter: 21961
Counter: 21962
Counter: 21963
Counter: 21964
Counter: 21965
Counter: 2

Counter: 22454
Counter: 22455
Counter: 22456
Counter: 22457
Counter: 22458
Counter: 22459
Counter: 22460
Counter: 22461
Counter: 22462
Counter: 22463
Counter: 22464
Counter: 22465
Counter: 22466
Counter: 22467
Counter: 22468
Counter: 22469
Counter: 22470
Counter: 22471
Counter: 22472
Counter: 22473
Counter: 22474
Counter: 22475
Counter: 22476
Counter: 22477
Counter: 22478
Counter: 22479
Counter: 22480
Counter: 22481
Counter: 22482
Counter: 22483
Counter: 22484
Counter: 22485
Counter: 22486
Counter: 22487
Counter: 22488
Counter: 22489
Counter: 22490
Counter: 22491
Counter: 22492
Counter: 22493
Counter: 22494
Counter: 22495
Counter: 22496
Counter: 22497
Counter: 22498
Counter: 22499
Counter: 22500
Counter: 22501
Counter: 22502
Counter: 22503
Counter: 22504
Counter: 22505
Counter: 22506
Counter: 22507
Counter: 22508
Counter: 22509
Counter: 22510
Counter: 22511
Counter: 22512
Counter: 22513
Counter: 22514
Counter: 22515
Counter: 22516
Counter: 22517
Counter: 22518
Counter: 22519
Counter: 2

Counter: 23001
Counter: 23002
Counter: 23003
Counter: 23004
Counter: 23005
Counter: 23006
Counter: 23007
Counter: 23008
Counter: 23009
Counter: 23010
Counter: 23011
Counter: 23012
Counter: 23013
Counter: 23014
Counter: 23015
Counter: 23016
Counter: 23017
Counter: 23018
Counter: 23019
Counter: 23020
Counter: 23021
Counter: 23022
Counter: 23023
Counter: 23024
Counter: 23025
Counter: 23026
Counter: 23027
Counter: 23028
Counter: 23029
Counter: 23030
Counter: 23031
Counter: 23032
Counter: 23033
Counter: 23034
Counter: 23035
Counter: 23036
Counter: 23037
Counter: 23038
Counter: 23039
Counter: 23040
Counter: 23041
Counter: 23042
Counter: 23043
Counter: 23044
Counter: 23045
Counter: 23046
Counter: 23047
Counter: 23048
Counter: 23049
Counter: 23050
Counter: 23051
Counter: 23052
Counter: 23053
Counter: 23054
Counter: 23055
Counter: 23056
Counter: 23057
Counter: 23058
Counter: 23059
Counter: 23060
Counter: 23061
Counter: 23062
Counter: 23063
Counter: 23064
Counter: 23065
Counter: 23066
Counter: 2

Counter: 23570
Counter: 23571
Counter: 23572
Counter: 23573
Counter: 23574
Counter: 23575
Counter: 23576
Counter: 23577
Counter: 23578
Counter: 23579
Counter: 23580
Counter: 23581
Counter: 23582
Counter: 23583
Counter: 23584
Counter: 23585
Counter: 23586
Counter: 23587
Counter: 23588
Counter: 23589
Counter: 23590
Counter: 23591
Counter: 23592
Counter: 23593
Counter: 23594
Counter: 23595
Counter: 23596
Counter: 23597
Counter: 23598
Counter: 23599
Counter: 23600
Counter: 23601
Counter: 23602
Counter: 23603
Counter: 23604
Counter: 23605
Counter: 23606
Counter: 23607
Counter: 23608
Counter: 23609
Counter: 23610
Counter: 23611
Counter: 23612
Counter: 23613
Counter: 23614
Counter: 23615
Counter: 23616
Counter: 23617
Counter: 23618
Counter: 23619
Counter: 23620
Counter: 23621
Counter: 23622
Counter: 23623
Counter: 23624
Counter: 23625
Counter: 23626
Counter: 23627
Counter: 23628
Counter: 23629
Counter: 23630
Counter: 23631
Counter: 23632
Counter: 23633
Counter: 23634
Counter: 23635
Counter: 2

Counter: 24130
Counter: 24131
Counter: 24132
Counter: 24133
Counter: 24134
Counter: 24135
Counter: 24136
Counter: 24137
Counter: 24138
Counter: 24139
Counter: 24140
Counter: 24141
Counter: 24142
Counter: 24143
Counter: 24144
Counter: 24145
Counter: 24146
Counter: 24147
Counter: 24148
Counter: 24149
Counter: 24150
Counter: 24151
Counter: 24152
Counter: 24153
Counter: 24154
Counter: 24155
Counter: 24156
Counter: 24157
Counter: 24158
Counter: 24159
Counter: 24160
Counter: 24161
Counter: 24162
Counter: 24163
Counter: 24164
Counter: 24165
Counter: 24166
Counter: 24167
Counter: 24168
Counter: 24169
Counter: 24170
Counter: 24171
Counter: 24172
Counter: 24173
Counter: 24174
Counter: 24175
Counter: 24176
Counter: 24177
Counter: 24178
Counter: 24179
Counter: 24180
Counter: 24181
Counter: 24182
Counter: 24183
Counter: 24184
Counter: 24185
Counter: 24186
Counter: 24187
Counter: 24188
Counter: 24189
Counter: 24190
Counter: 24191
Counter: 24192
Counter: 24193
Counter: 24194
Counter: 24195
Counter: 2

In [4]:
images = np.array(images)
landmarks = np.array(landmarks)
visibility = np.array(visibility)
pose = np.array(pose)
gender = np.array(gender)

x_train, x_test, y_train_landmarks, y_test_landmarks, y_train_visibility, y_test_visibility, y_train_pose, y_test_pose, \
y_train_gender, y_test_gender = train_test_split(images, landmarks, visibility, pose, gender, test_size = 0.20, shuffle = True)

x_train = x_train.reshape((x_train.shape[0], 227, 227, 1))
x_test = x_test.reshape((x_test.shape[0], 227, 227, 1))

x_train = x_train/255
x_test = x_test/255

print(x_train.shape, y_train_landmarks.shape)
print(x_train.shape, y_train_visibility.shape)
print(x_train.shape, y_train_pose.shape)
print(x_train.shape, y_train_gender.shape)
print(x_train.shape, y_test_landmarks.shape)
print(x_train.shape, y_test_visibility.shape)
print(x_test.shape, y_test_pose.shape)
print(x_test.shape, y_test_gender.shape)

(19507, 227, 227, 1) (19507, 42)
(19507, 227, 227, 1) (19507, 21)
(19507, 227, 227, 1) (19507, 3)
(19507, 227, 227, 1) (19507,)
(19507, 227, 227, 1) (4877, 42)
(19507, 227, 227, 1) (4877, 21)
(4877, 227, 227, 1) (4877, 3)
(4877, 227, 227, 1) (4877,)


In [10]:
def HyperFace():
    input_layer = Input(shape = (227, 227, 1), name = 'input_layer')

    conv1 = Conv2D(96, (11,11), strides = 4, activation = 'relu', padding = 'valid', name = 'conv1')(input_layer)
    max1 = MaxPool2D((3,3), strides = 2, padding = 'valid',name = 'max1')(conv1)

    conv1a = Conv2D(256, (4,4), strides = 4, activation = 'relu', padding = 'valid', name = 'conv1a')(max1)

    conv2 = Conv2D(256, (5,5), strides = 1, activation = 'relu', padding = 'same', name = 'conv2')(max1)
    max2 = MaxPool2D((3,3), strides = 2, padding = 'valid', name = 'max2')(conv2)
    conv3 = Conv2D(384, (3,3), strides = 1, activation = 'relu', padding = 'same', name = 'conv3')(max2)

    conv3a = Conv2D(256, (2,2), strides = 2, activation = 'relu', padding = 'valid', name = 'conv3a')(conv3)

    conv4 = Conv2D(384, (3,3), strides = 1, activation = 'relu', padding = 'same', name = 'conv4')(conv3)
    conv5 = Conv2D(256, (3,3), strides = 1, activation = 'relu', padding = 'same', name = 'conv5')(conv4)
    pool5 = MaxPool2D((3,3), strides = 2, padding = 'valid', name = 'pool5')(conv5)

    concat = concatenate([conv1a, conv3a, pool5], name = 'concat')

    conv_all = Conv2D(192, (1,1), strides = 1, activation = 'relu', padding = 'valid', name = 'conv_all')(concat)
    flatten = Flatten(name = 'flatten')(conv_all)

    fc_full = Dense(3072, activation = 'relu', name = 'fc_full')(flatten)

    #fc_detection = Dense(512, activation = 'relu', name = 'fc_detection')(fc_full)
    fc_landmarks = Dense(512, activation = 'relu', name = 'fc_landmarks')(fc_full)
    fc_visibility = Dense(512, activation = 'relu', name = 'fc_visibility')(fc_full)
    fc_pose = Dense(512, activation = 'relu', name = 'fc_pose')(fc_full)
    fc_gender = Dense(512, activation = 'relu', name = 'fc_gender')(fc_full)

    #out_detection = Dense(1, activation = 'softmax', name = 'out_detection')(fc_detection)
    out_landmarks = Dense(42, activation = 'softmax', name = 'out_landmarks')(fc_landmarks)
    out_visibility = Dense(21, activation = 'sigmoid', name = 'out_visibility')(fc_visibility)
    out_pose = Dense(3, activation = 'softmax', name = 'out_pose')(fc_pose)
    out_gender = Dense(1, activation = 'sigmoid', name = 'out_gender')(fc_gender)

    model = Model(inputs = input_layer, outputs = [out_landmarks, out_visibility, out_pose, out_gender])#out_detection 
    #plot_model(model, to_file='model.png', show_shapes = True, show_layer_names = True, rankdir = 'TB')
    model.summary()
    
    return model

model = HyperFace()

losses = {
    #"out_detection" : "binary_crossentropy",
    "out_landmarks" : "mean_squared_error",
    "out_visibility" : "mean_squared_error",
    "out_pose" : "mean_squared_error",
    "out_gender" : "binary_crossentropy"
}

loss_weights = {
    #"out_detection" : 1.0,
    "out_landmarks" : 1.0,
    "out_visibility" : 1.0,
    "out_pose" : 1.0,
    "out_gender" : 1.0
}

optimizer = RMSprop(
    lr=0.01,
    rho=0.9,
    epsilon=None,
    decay=0.0
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.00001)

model.compile(optimizer = optimizer, loss = losses, loss_weights = loss_weights, metrics = ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 227, 227, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 55, 55, 96)   11712       input_layer[0][0]                
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 27, 27, 96)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 27, 27, 256)  614656      max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [ ]:
model.fit(x_train, { "out_landmarks" : y_train_landmarks, "out_visibility" : y_train_visibility, "out_pose" : y_train_pose, "out_gender" : y_train_gender},
         validation_data = (x_test, {"out_landmarks" : y_test_landmarks, "out_visibility" : y_test_visibility,  "out_pose" : y_test_pose, "out_gender" : y_test_gender}),
         epochs = 30,
         verbose = 1, 
         batch_size = 64, 
         callbacks = [reduce_lr]
         )